# Đọc dữ liệu

In [1]:
import pandas as pd
san_pham=pd.read_csv('product_catalog_clean.csv')
dat_hang=pd.read_csv('order_details_clean.csv')
tra_lai=pd.read_csv('return_refund_clean.csv')

# 1.Tính doanh thu theo sản phẩm (quantity × unit_price)

In [2]:
dat_hang['doanh_thu']=dat_hang['quantity']*dat_hang['unit_price']

doanh_thu_sp=(
    dat_hang
    .merge(san_pham[["product_id", "product_name"]], on="product_id", how="left")
    .groupby(['product_id','product_name'],as_index=False)['doanh_thu']
    .sum()
)
print('Doanh thu theo sản phẩm')
print(doanh_thu_sp)

Doanh thu theo sản phẩm
  product_id        product_name   doanh_thu
0       PR01  tai nghe bluetooth  22700000.0
1       PR02       loa bluetooth  26300000.0
2       PR03         ban phim co  10450000.0
3       PR04     chuot khong day  18100000.0
4       PR05        sac du phong  20400000.0


# 2.Tính doanh thu trung bình theo danh mục.

In [3]:
doanh_thu_theo_danh_muc=(
    dat_hang
    .merge(san_pham[['product_id','category']],on='product_id',how='left')
    .groupby('category',as_index=False)['doanh_thu']
    .mean()
)
print('\n Doanh thu trung bình theo mục')
print(doanh_thu_theo_danh_muc)


 Doanh thu trung bình theo mục
   category      doanh_thu
0  gia dụng  712962.962963
1   điện tử  900757.575758


# 3.Tổng hợp tỷ lệ trả hàng theo sản phẩm.

In [4]:
so_don_ban=(
    dat_hang
    .groupby('product_id')
    .size()
    .reset_index(name='so_don_ban')
    )
so_don_tra=(
    tra_lai
    .groupby('product_id')
    .size()
    .reset_index(name='so_don_tra')
)

ty_le_tra=(
    so_don_ban
    .merge(so_don_tra,on='product_id',how='left')
    .fillna(0)
)

ty_le_tra['ty_le_tra']=ty_le_tra['so_don_tra']/ty_le_tra['so_don_ban']
print("\nTỷ lệ trả hàng theo sản phẩm:")
print(ty_le_tra)


Tỷ lệ trả hàng theo sản phẩm:
  product_id  so_don_ban  so_don_tra  ty_le_tra
0       PR01          22           4   0.181818
1       PR02          24          14   0.583333
2       PR03          20           8   0.400000
3       PR04          26          18   0.692308
4       PR05          28          16   0.571429


# 4.Xác định nhóm sản phẩm có hiệu suất thấp.

In [5]:
hieu_suat=(
    doanh_thu_sp
    .merge(ty_le_tra,on='product_id',how='left')
    .fillna(0)
)
dt_tb = hieu_suat["doanh_thu"].mean()
tl_tra_tb = hieu_suat["ty_le_tra"].mean()
san_pham_hieu_suat_thap = hieu_suat[
    (hieu_suat["doanh_thu"] < dt_tb) &
    (hieu_suat["ty_le_tra"] > tl_tra_tb)
]
print("\nNhóm sản phẩm có hiệu suất thấp:")
print(san_pham_hieu_suat_thap)


Nhóm sản phẩm có hiệu suất thấp:
  product_id     product_name   doanh_thu  so_don_ban  so_don_tra  ty_le_tra
3       PR04  chuot khong day  18100000.0          26          18   0.692308


# 5.Tính doanh thu theo tháng.

In [6]:
dat_hang["order_date"] = pd.to_datetime(dat_hang["order_date"])
doanh_thu_thang = (
    dat_hang
    .groupby(dat_hang["order_date"].dt.to_period("M"))["doanh_thu"]
    .sum()
    .reset_index()
)
doanh_thu_thang["order_date"] = doanh_thu_thang["order_date"].astype(str)

print("\n Doanh thu theo tháng:")
print(doanh_thu_thang)


 Doanh thu theo tháng:
  order_date   doanh_thu
0    2024-01  24350000.0
1    2024-02  30200000.0
2    2024-03  32000000.0
3    2024-04  11400000.0
